# lmport Raw Data

In [ ]:
import pandas as pd
import numpy as np
import datetime as datetime
import time
import math
from dateutil.parser import parse

In [ ]:
alldata=pd.read_csv("raw_data.txt", sep='|')

# Delete Features and Add Head

In [ ]:
def delete_and_add_head(alldata):
# delete constant feature
    del alldata[16]
    del alldata[48]
    del alldata[60]
    del alldata[62]

    head=['auth_id','acct_id','fraud_ind','acct_actvn_dt','acct_avl_cash_before_amt','acct_avl_money_before_amt','acct_credit_limit','acct_curr_balance','acct_multicard_ind','acct_open_dt','acct_prod_cd','acct_type_cd','address_vfcn_frmt_cd','address_vfcn_respns_cd','apprd_authzn_cnt','apprd_cash_authzn_cnt','authzn_acct_stat_cd','authzn_amount','authzn_type_cd','authzn_char_cd','authzn_opset_id','authzn_orig_src_id','authzn_outstd_amt','authzn_outstd_cash_amt','authzn_rqst_proc_cd','authzn_rqst_proc_dt','authzn_rqst_proc_tm','authzn_rqst_type_cd','authzn_trmnl_pin_capblt_num','avg_dly_authzn_amt','card_vfcn_2_respns_cd','card_vfcn_2_vldtn_dur','card_vfcn_msmt_reas_cd','card_vfcn_presnc_cd','card_vfcn_respns_cd','card_vfcn2_vldtn_cd','cardholder_present_cd','currency_rate','elctr_cmrc_ind_cd','home_phn_num_chng_dur','hotel_stay_car_rentl_dur','last_adr_chng_dur','last_plstc_rqst_reas_cd','mrch_type_cd','mrch_country_cd','new_user_added_dur','phn_chng_snc_appn_ind','pin_vldtn_ind','plstc_actvn_dt','plstc_actvn_reqd_ind','plstc_frst_use_ts','plstc_isu_dur','plstc_prev_curr_cd','plstc_rqst_ts','pos_cond_cd','pos_entry_mthd_cd','rcurg_authzn_ind','rvrsl_ind','src_crcy_cd','trmnl_attndnc_cd','trmnl_capblt_cd','trmnl_clasfn_cd','trmnl_id','trmnl_pin_capblt_cd','distance_from_home']
    alldata.columns=head

    #delete non-sense feature
    del alldata['trmnl_id']

    #delete skewed and non significance features
    del alldata['acct_type_cd']
    del alldata['authzn_opset_id']
    del alldata['authzn_rqst_proc_cd']
    del alldata['card_vfcn_msmt_reas_cd']
    del alldata['phn_chng_snc_appn_ind']
    del alldata['arqc_rslt_cd']
    del alldata['pin_blk_cd']
    del alldata['sendr_rsidnl_cntry_cd']
    del alldata['src_crcy_dcml_psn_num']
    del alldata['trmnl_id']

    print alldata.shape

delete_and_add_head(alldata)

# Get Angles

In [ ]:
alldata['authzn_rqst_proc_tm'] = alldata.authzn_rqst_proc_tm.apply(lambda x: x.rsplit('.', 1)[0])
dac=np.vstack(alldata.authzn_rqst_proc_tm.apply(lambda x: list(map(int, x.split(':')))).values)

#def angle(x):
import math
angel=[]
for i in range(0,len(dac)):
    an=((dac[i][0]+dac[i][1]/60.0)/24.0)*2*math.pi
    angel.append(an)

alldata['angel']=angel

# Transform authurization request date and time

In [ ]:
# Combine authurization request date and time, and transform them to seconds since 2013,07,29.
def transform_date(x):
    x=parse(x)
    x=time.mktime(x.timetuple())
    return x             
                     
alldata["authzn_rqst_time"] = alldata["authzn_rqst_proc_dt"].map(str) +' '+ alldata["authzn_rqst_proc_tm"]
alldata['authzn_rqst_time'] = alldata.authzn_rqst_time.apply(transform_date)

In [ ]:
# sort alldata by acct_id and authzn_rqst_time
alldata=alldata.sort(['acct_id','authzn_rqst_time'],ascending=[1,0])
alldata=alldata.reset_index(drop=True) 

In [ ]:
def morning(x):
    if x>=8 and x<=20:
        x='m'
    else:
        x='n'
    return x
def get_m(alldata):
    alldata['authzn_rqst_proc_tm'] = alldata.authzn_rqst_proc_tm.apply(lambda x: x.rsplit('.', 1)[0])
    dac = np.vstack(alldata.authzn_rqst_proc_tm.apply(lambda x: list(map(int, x.split(':')))).values)
    alldata['authzn_rqst_hour'] = dac[:,0]
    alldata['authzn_rqst_hour']=alldata.authzn_rqst_hour.apply(morning)
    
get_m(alldata)

del alldata["authzn_rqst_proc_tm"]
del alldata["authzn_rqst_proc_dt"]

# Velocity Feature 1: Time Feature

## get tuple, unique id sorted

In [ ]:
alldata['tuple']=zip(alldata['acct_id'],alldata['authzn_rqst_time'],alldata['angel'])
uniqueid=alldata['acct_id'].value_counts()
a1=np.unique(uniqueid.index.get_values()).tolist()
group=alldata.groupby('acct_id')
print len(a1),alldata.shape

## function

In [ ]:
def time_feature(alldata,period):
    time_feature4=[]
    time_feature5=[]
    time_feature6=[]
    f=0
    for e in range(500,2001,500): # seperate into small dataset
        
        if e==2000:
            e=len(a1)-1
            new=alldata[(alldata['acct_id']>=a1[f]) & (alldata['acct_id']<=a1[e])]

        else:
            new=alldata[(alldata['acct_id']>=a1[f]) & (alldata['acct_id']<a1[e])]
        
        
        f=e
        uniqueid=new['acct_id'].value_counts()
        a2=np.unique(uniqueid.index.get_values()).tolist()
        group=new.groupby('acct_id')



        for i in a2:
            ti=group.get_group(i).tuple.as_matrix()
            l=len(ti)
            if l==1:
                time_feature4.append('N')
                time_feature5.append('N')
                time_feature6.append('N')
                continue
            elif l==2:
                time_feature4.extend(['N','N'])
                time_feature5.extend(['N','N'])
                time_feature6.extend(['N','N'])
                continue
            else:
                j=0 #from the latest transaction 
                while j < l: #while transaction in the same id group


                    if j==l-2:
                        time_feature4.extend(['N','N'])
                        time_feature5.extend(['N','N'])
                        time_feature6.extend(['N','N'])
                        break

                    else:
                        an_sum_cos=0.0
                        an_sum_sin=0.0
                        an_con_cos=0.0
                        an_con_sin=0.0

                        t1=ti[j][1]-86400*period # 7days ago interval 
                        p=j+1 # next transaction

                        while p < l:
                            if ti[p][1]>=t1:
                                an_sum_cos=an_sum_cos+math.cos(ti[p][2])
                                an_con_cos=an_con_cos+1
                                an_sum_sin=an_sum_sin+math.sin(ti[p][2])
                                an_con_sin=an_con_sin+1
                                p=p+1
                            else:
                                break
                        if an_con_cos==0.0:
                            time_feature4.append('N')
                            time_feature5.append('N')
                            time_feature6.append('N')


                        elif an_con_cos==1.0:
                            time_feature4.append('N')
                            time_feature5.append('N')
                            time_feature6.append('N')


                        elif an_con_cos>1.0:
                            cp=an_sum_cos/an_con_cos
                            sp=an_sum_sin/an_con_sin
                            if (cp>0)&(sp>0):
                                tp=math.atan(sp/cp)
                            elif cp<0:
                                tp=math.atan(sp/cp)+math.pi
                            elif (sp<0)&(cp>0):
                                tp=math.atan(sp/cp)+2*math.pi

                            
                            ang4 = (4.0/24.0)*2.0*math.pi
                            if ((tp-ang4)<ti[j][2])&((tp+ang4)>ti[j][2]):
                                time_feature4.append('T')
                            else:
                                time_feature4.append('F')

                            
                            ang5 =(5.0/24.0)*2.0*math.pi
                            if ((tp-ang5)<ti[j][2])&((tp+ang5)>ti[j][2]):
                                time_feature5.append('T')
                            else:
                                time_feature5.append('F')

                            
                            
                            ang6 = (6.0/24.0)*2.0*math.pi
                            if ((tp-ang6)<ti[j][2])&((tp+ang6)>ti[j][2]):
                                time_feature6.append('T')
                            else:
                                time_feature6.append('F')

                    j=j+1
        print e
    return time_feature4,time_feature5,time_feature6

In [ ]:
start_time = time.time()
t74,t75,t76=time_feature(alldata,7)
t154,t155,t156=time_feature(alldata,15)
t304,t305,t306=time_feature(alldata,30)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
v1=[t74,t75,t76,t154,t155,t156,t304,t305,t306]
v11=np.array(v1)
v111=np.transpose(v11)
np.savetxt("f3.txt", v111, delimiter="|",fmt='%s')

In [ ]:
test=pd.read_csv('f3.txt',sep='|',header=None)
cols=[]
for i in [7,15,30]:
    cols.append(str(i)+'days_4hour')
    cols.append(str(i)+'days_5hour')
    cols.append(str(i)+'days_6hour')
test.columns = cols
test.head()

In [ ]:
alldatanew=pd.concat([alldata,test],axis=1)
alldatanew.to_csv('valocity_feature_1_added_train.txt',sep='|',index=None)

# Velocity Feature 2: 
## Add transaction counts and amounts for the same ID in different time periods: 1,3,6,12,18,24,72,168 hours

In [ ]:
# prepare
alldata['tuple']=zip(alldata['acct_id'],alldata['authzn_rqst_time'],alldata['authzn_amount'])
uniqueid=alldata['acct_id'].value_counts()
a1=np.unique(uniqueid.index.get_values()).tolist()
group=alldata.groupby('acct_id')
print len(a1),alldata.shape

In [ ]:
mysum1=[]
mysum2=[]
mysum3=[]
mysum4=[]
mysum5=[]
mysum6=[]
mysum7=[]
mysum8=[]
mycounts1=[]
mycounts2=[]
mycounts3=[]
mycounts4=[]
mycounts5=[]
mycounts6=[]
mycounts7=[]
mycounts8=[]
f=0       
for e in range(500,2001,500): # seperate into small dataset
        
    if e==2000:
        e=len(a1)-1
        new=alldata[(alldata['acct_id']>=a1[f]) & (alldata['acct_id']<=a1[e])]

    else:
        new=alldata[(alldata['acct_id']>=a1[f]) & (alldata['acct_id']<a1[e])]
    
    
    f=e
    uniqueid=new['acct_id'].value_counts()
    a2=np.unique(uniqueid.index.get_values()).tolist()
    group=new.groupby('acct_id')

    for i in a2:
        ti=group.get_group(i).tuple.as_matrix()
        l=len(ti)
        #ti=groupi.tuple
        j=0
        while j < l:
            sum=0.0
            counts=0
            sum1=0.0
            counts1=0
            sum2=0.0
            counts2=0
            sum3=0.0
            counts3=0
            sum4=0.0
            counts4=0
            sum5=0.0
            counts5=0
            sum6=0.0
            counts6=0
            sum7=0.0
            counts7=0
            sum8=0.0
            counts8=0

            
            p=j+1

            if p == l:
                mysum1.append(sum)
                mycounts1.append(counts)
                mysum2.append(sum)
                mycounts2.append(counts)
                mysum3.append(sum)
                mycounts3.append(counts)
                mysum4.append(sum)
                mycounts4.append(counts)
                mysum5.append(sum)
                mycounts5.append(counts)
                mysum6.append(sum)
                mycounts6.append(counts)
                mysum7.append(sum)
                mycounts7.append(counts)
                mysum8.append(sum)
                mycounts8.append(counts)
                break

            else:
                t1=ti[j][1]-3600*1
                #print ti[j][1]
                t2=ti[j][1]-3600*3
                t3=ti[j][1]-3600*6
                t4=ti[j][1]-3600*12
                t5=ti[j][1]-3600*18
                t6=ti[j][1]-3600*24
                t7=ti[j][1]-3600*72
                t8=ti[j][1]-3600*168
                while p < l:
                    if ti[p][1]>=t1:
                        sum=sum+ti[p][2]
                        counts=counts+1
                        p=p+1
                    else:
                        break
                mysum1.append(sum)
                mycounts1.append(counts)





                if p == l: 
                    mysum2.append(sum)
                    mycounts2.append(counts)
                else:
                    while p<l:
                        if ti[p][1]>=t2:
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum2.append(sum)
                    mycounts2.append(counts)



                if p == l: 
                    mysum3.append(sum)
                    mycounts3.append(counts)
                else:
                    while p < l:
                        if ti[p][1]>=t3:
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum3.append(sum)
                    mycounts3.append(counts)




                if p == l: 
                    mysum4.append(sum)
                    mycounts4.append(counts)
                else:
                    while p < l:
                        if ti[p][1]>=t4:
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum4.append(sum)
                    mycounts4.append(counts)




                if p == l: 
                    mysum5.append(sum)
                    mycounts5.append(counts)
                else:
                    while p < l:
                        if ti[p][1]>=t5:
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum5.append(sum)
                    mycounts5.append(counts)




                if p == l: 
                    mysum6.append(sum)
                    mycounts6.append(counts)
                else:
                    while p < l:
                        if ti[p][1]>=t6:
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum6.append(sum)
                    mycounts6.append(counts)




                if p == l: 
                    mysum7.append(sum)
                    mycounts7.append(counts)
                else:
                    while p < l:
                        if ti[p][1]>=t7:
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum7.append(sum)
                    mycounts7.append(counts)


                if p == l: 
                    mysum8.append(sum)
                    mycounts8.append(counts)
                else:
                    while p < l:
                        if ti[p][1]>=t8:
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum8.append(sum)
                    mycounts8.append(counts)



            j=j+1
    print e   

# output

In [ ]:
v1=[mysum1,mycounts1,mysum2,mycounts2,mysum3,mycounts3,mysum4,mycounts4,mysum5,mycounts5,mysum6,mycounts6,mysum7,mycounts7,mysum8,mycounts8]
v11=np.array(v1)
v111=np.transpose(v11)
np.savetxt("f1.txt", v111, delimiter="|",fmt='%s')

In [ ]:
test2=pd.read_csv('f1.txt',sep='|',header=None)
cols=[]
for i in [1,3,6,12,18,24,72,168]:
    cols.append('sum'+str(i)+'h')
    cols.append('counts'+str(i)+'h')
test2.columns = cols
alldatanew2=pd.concat([alldatanew,test2],axis=1)

In [ ]:
alldatanew2.to_csv('valocity_feature_2_added_train.txt',sep='|',index=None)

# Velocity Feature 3
## Count transaction times and amount with same merchant country and merchant type, for each ID in different time periods: 1,3,6,12,18,24,72,168 hours

In [ ]:
# group merchant type
def transform_merchant_type(x):
    if x<2000:
        x = "con_service"
    elif x>=2000 and x<3000:
        x="wholesale"
    elif x>=3000 and x<=3350:
        x="airline"
    elif x>=3351 and x<=3500:
        x="car_rental"   
    elif x>=3501 and x<=4000:
        x="hotel"
    elif x>=4001 and x<4812:
        x="transportation"
    elif x>=4812 and x<5000:
        x="utilities"
    elif x>=5000 and x<=5199:
        x= "wholesale"
    elif x>=5200 and x<5500:
        x="retail"
    elif x>=5500 and x<5600:
        x="automobiles"
    elif x>=5600 and x<5700:
        x="clothing"
    elif x>=5700 and x<6000:
        x="miscellaneous"
    elif x>=6000 and x<7210:
        x= "service_provider"
    elif x>=7210 and x<7311:
        x="personal_service"
    elif x>=7311 and x<7531:
        x="business"
    elif x>=7531 and x<7829:
        x="repair"
    elif x>=7829 and x<8000:
        x="amusement"
    elif x>=8000 and x<9000:
        x="professional_service"
    elif x>=9000:
        x="government"
    elif x==9401:
        x="other"
    elif x==7011:
        x="hotel"
    elif x>=4511:
        x="airline"
    elif x==7512:
        x="car_rental"
    elif x in [7375,7379,7829,8734]:
        x="wholesale"
    elif x in [4829,6050,6051,6529,6530,6534]:
        x="quasi_cash"
    
    return x

In [ ]:
# group countries
def transform_country(x):    
    if x in [660,28,533,44,52,535,92,136,192,531,212,214,308,312,332,388,474,500,630,652,659,662,663,670,534,780,796,850,84,188,222,320,340,484,558,591,32,68,76,152,170,218,238,254,328,600,604,740,858,862]:
        x="America"
    elif x ==840:
        x="USA"
    elif x in [398,417,762,795,860,51,31,48,196,268,368,376,400,414,422,275,512,634,682,760,792,784,887,96,116,360,418,458,104,608,702,764,626,704,4,50,64,356,364,462,524,586,144,156,344,446,392,408,410,496,158]:
        x="Asia"
    elif x in [40,56,250,276,280,438,442,492,528,756,8,20,70,191,292,300,336,380,807,470,499,620,674,688,705,724,248,830,208,233,234,246,831,352,372,832,428,440,833,578,744,752,826,112,100,203,348,498,616,642,643,703,804]:
        x="Europe"
    else:
        x="Others"
    return x

In [ ]:
def group_features(df):
    df['mrch_country_cd'] = df.mrch_country_cd.apply(transform_country)
    df['mrch_type_cd'] = df.mrch_type_cd.apply(transform_merchant_type)
group_features(alldata)

# Create Features

In [ ]:
alldata['tuple']=zip(alldata['acct_id'],alldata['authzn_rqst_time'],alldata['authzn_amount'],alldata['mrch_type_cd'],alldata['mrch_country_cd'])
uniqueid=alldata['acct_id'].value_counts()
a1=np.unique(uniqueid.index.get_values()).tolist()
group=alldata.groupby('acct_id')
print len(a1),alldata.shape

In [ ]:
mysum1=[]
mysum2=[]
mysum3=[]
mysum4=[]
mysum5=[]
mysum6=[]
mysum7=[]
mysum8=[]
mycounts1=[]
mycounts2=[]
mycounts3=[]
mycounts4=[]
mycounts5=[]
mycounts6=[]
mycounts7=[]
mycounts8=[]
f=0       
for e in range(500,2001,500): # seperate into small dataset
        
    if e==2000:
        e=len(a1)-1
        new=alldata[(alldata['acct_id']>=a1[f]) & (alldata['acct_id']<=a1[e])]

    else:
        new=alldata[(alldata['acct_id']>=a1[f]) & (alldata['acct_id']<a1[e])]
    
    
    f=e
    uniqueid=new['acct_id'].value_counts()
    a2=np.unique(uniqueid.index.get_values()).tolist()
    group=new.groupby('acct_id')
    for i in a2:
        ti=group.get_group(i).tuple.as_matrix()
        l=len(ti)
        #ti=groupi.tuple
        j=0
        while j < l:
            sum=0.0
            counts=0
            sum1=0.0
            counts1=0
            sum2=0.0
            counts2=0
            sum3=0.0
            counts3=0
            sum4=0.0
            counts4=0
            sum5=0.0
            counts5=0
            sum6=0.0
            counts6=0
            sum7=0.0
            counts7=0
            sum8=0.0
            counts8=0

            t1=ti[j][1]-3600*1
            #print ti[j][1]
            t2=ti[j][1]-3600*3
            t3=ti[j][1]-3600*6
            t4=ti[j][1]-3600*12
            t5=ti[j][1]-3600*18
            t6=ti[j][1]-3600*24
            t7=ti[j][1]-3600*72
            t8=ti[j][1]-3600*168
            mty=ti[j][3]
            mcon=ti[j][4]
            p=j+1

            if p == l:
                mysum1.append(sum)
                mycounts1.append(counts)
                mysum2.append(sum)
                mycounts2.append(counts)
                mysum3.append(sum)
                mycounts3.append(counts)
                mysum4.append(sum)
                mycounts4.append(counts)
                mysum5.append(sum)
                mycounts5.append(counts)
                mysum6.append(sum)
                mycounts6.append(counts)
                mysum7.append(sum)
                mycounts7.append(counts)
                mysum8.append(sum)
                mycounts8.append(counts)
                break

            else:
                while p < l:
                    if (ti[p][1]>=t1) &(ti[p][3]==mty)&(ti[p][4]==mcon):
                        sum=sum+ti[p][2]
                        counts=counts+1
                        p=p+1
                    else:
                        break
                mysum1.append(sum)
                mycounts1.append(counts)





                if p == l: 
                    mysum2.append(sum)
                    mycounts2.append(counts)
                else:
                    while p<l:
                        if (ti[p][1]>=t2)&(ti[p][3]==mty)&(ti[p][4]==mcon):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum2.append(sum)
                    mycounts2.append(counts)



                if p == l: 
                    mysum3.append(sum)
                    mycounts3.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t3) &(ti[p][3]==mty)&(ti[p][4]==mcon):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum3.append(sum)
                    mycounts3.append(counts)




                if p == l: 
                    mysum4.append(sum)
                    mycounts4.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t4) &(ti[p][3]==mty)&(ti[p][4]==mcon):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum4.append(sum)
                    mycounts4.append(counts)




                if p == l: 
                    mysum5.append(sum)
                    mycounts5.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t5) &(ti[p][3]==mty)&(ti[p][4]==mcon):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum5.append(sum)
                    mycounts5.append(counts)




                if p == l: 
                    mysum6.append(sum)
                    mycounts6.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t6) &(ti[p][3]==mty)&(ti[p][4]==mcon):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum6.append(sum)
                    mycounts6.append(counts)




                if p == l: 
                    mysum7.append(sum)
                    mycounts7.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t7) &(ti[p][3]==mty)&(ti[p][4]==mcon):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum7.append(sum)
                    mycounts7.append(counts)


                if p == l: 
                    mysum8.append(sum)
                    mycounts8.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t8) &(ti[p][3]==mty)&(ti[p][4]==mcon):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum8.append(sum)
                    mycounts8.append(counts)



            j=j+1
    print e

In [ ]:
v1=[mysum1,mycounts1,mysum2,mycounts2,mysum3,mycounts3,mysum4,mycounts4,mysum5,mycounts5,mysum6,mycounts6,mysum7,mycounts7,mysum8,mycounts8]
v11=np.array(v1)
v111=np.transpose(v11)
np.savetxt("f2.txt", v111, delimiter="|",fmt='%s')

In [ ]:
test3=pd.read_csv('f2.txt',sep='|',header=None)
cols=[]
for i in [1,3,6,12,18,24,72,168]:
    cols.append('mt_sum'+str(i)+'h')
    cols.append('mt_counts'+str(i)+'h')
test3.columns = cols
alldatanew3=pd.concat([alldatanew2,test3],axis=1)

In [ ]:
alldatanew3.to_csv('valocity_feature_3_added_train.txt',sep='|',index=None)

# Velocity Feature 4
## Count transaction times and amount with the same merchant type and pos entry mode, for each ID in different time periods: 1,3,6,12,18,24,72,168 hours

In [ ]:
alldata['tuple']=zip(alldata['acct_id'],alldata['authzn_rqst_time'],alldata['authzn_amount'],alldata['mrch_type_cd'],alldata['pos_entry_mthd_cd'])
uniqueid=alldata['acct_id'].value_counts()
a1=np.unique(uniqueid.index.get_values()).tolist()
group=alldata.groupby('acct_id')
print len(a1),alldata.shape

In [ ]:
mysum1=[]
mysum2=[]
mysum3=[]
mysum4=[]
mysum5=[]
mysum6=[]
mysum7=[]
mysum8=[]
mycounts1=[]
mycounts2=[]
mycounts3=[]
mycounts4=[]
mycounts5=[]
mycounts6=[]
mycounts7=[]
mycounts8=[]
f=0       
for e in range(5000,805001,5000): # seperate into small dataset
        
    if e==805000:
        e=len(a1)-1
        new=alldata[(alldata['acct_id']>=a1[f]) & (alldata['acct_id']<=a1[e])]

    else:
        new=alldata[(alldata['acct_id']>=a1[f]) & (alldata['acct_id']<a1[e])]
    
    
    f=e
    uniqueid=new['acct_id'].value_counts()
    a2=np.unique(uniqueid.index.get_values()).tolist()
    group=new.groupby('acct_id')

    for i in a2:
        ti=group.get_group(i).tuple.as_matrix()
        l=len(ti)
        j=0
        while j < l:
            sum=0.0
            counts=0
            sum1=0.0
            counts1=0
            sum2=0.0
            counts2=0
            sum3=0.0
            counts3=0
            sum4=0.0
            counts4=0
            sum5=0.0
            counts5=0
            sum6=0.0
            counts6=0
            sum7=0.0
            counts7=0
            sum8=0.0
            counts8=0

            t1=ti[j][1]-3600*1
            t2=ti[j][1]-3600*3
            t3=ti[j][1]-3600*6
            t4=ti[j][1]-3600*12
            t5=ti[j][1]-3600*18
            t6=ti[j][1]-3600*24
            t7=ti[j][1]-3600*72
            t8=ti[j][1]-3600*168
            mty=ti[j][3]
            em=ti[j][4]
            p=j+1

            if p == l:
                mysum1.append(sum)
                mycounts1.append(counts)
                mysum2.append(sum)
                mycounts2.append(counts)
                mysum3.append(sum)
                mycounts3.append(counts)
                mysum4.append(sum)
                mycounts4.append(counts)
                mysum5.append(sum)
                mycounts5.append(counts)
                mysum6.append(sum)
                mycounts6.append(counts)
                mysum7.append(sum)
                mycounts7.append(counts)
                mysum8.append(sum)
                mycounts8.append(counts)
                break

            else:
                while p < l:
                    if (ti[p][1]>=t1) &(ti[p][3]==mty)&(ti[p][4]==em):
                        sum=sum+ti[p][2]
                        counts=counts+1
                        p=p+1
                    else:
                        break
                mysum1.append(sum)
                mycounts1.append(counts)





                if p == l: 
                    mysum2.append(sum)
                    mycounts2.append(counts)
                else:
                    while p<l:
                        if (ti[p][1]>=t2)&(ti[p][3]==mty)&(ti[p][4]==em):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum2.append(sum)
                    mycounts2.append(counts)



                if p == l: 
                    mysum3.append(sum)
                    mycounts3.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t3) &(ti[p][3]==mty)&(ti[p][4]==em):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum3.append(sum)
                    mycounts3.append(counts)




                if p == l: 
                    mysum4.append(sum)
                    mycounts4.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t4) &(ti[p][3]==mty)&(ti[p][4]==em):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum4.append(sum)
                    mycounts4.append(counts)




                if p == l: 
                    mysum5.append(sum)
                    mycounts5.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t5) &(ti[p][3]==mty)&(ti[p][4]==em):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum5.append(sum)
                    mycounts5.append(counts)




                if p == l: 
                    mysum6.append(sum)
                    mycounts6.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t6) &(ti[p][3]==mty)&(ti[p][4]==em):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum6.append(sum)
                    mycounts6.append(counts)




                if p == l: 
                    mysum7.append(sum)
                    mycounts7.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t7) &(ti[p][3]==mty)&(ti[p][4]==em):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum7.append(sum)
                    mycounts7.append(counts)


                if p == l: 
                    mysum8.append(sum)
                    mycounts8.append(counts)
                else:
                    while p < l:
                        if (ti[p][1]>=t8) &(ti[p][3]==mty)&(ti[p][4]==em):
                            sum=sum+ti[p][2]
                            counts=counts+1
                            p=p+1
                        else:
                            break
                    mysum8.append(sum)
                    mycounts8.append(counts)



            j=j+1
    print e

In [ ]:
v1=[mysum1,mycounts1,mysum2,mycounts2,mysum3,mycounts3,mysum4,mycounts4,mysum5,mycounts5,mysum6,mycounts6,mysum7,mycounts7,mysum8,mycounts8]
v11=np.array(v1)
v111=np.transpose(v11)
v111.shape
np.savetxt("f22.txt", v111, delimiter="|",fmt='%s')

In [ ]:
test=pd.read_csv('f22.txt',sep='|',header=None)
cols=[]
for i in [1,3,6,12,18,24,72,168]:
    cols.append('sum'+str(i)+'h')
    cols.append('counts'+str(i)+'h')
test.columns = cols

test['acct_id']=alldata['acct_id']
test['auth_id']=alldata['auth_id']

In [ ]:
test.to_csv('valocity_feature_4_added_train.txt',sep='|',index=None)